Procedure
## Step 1: Coding a Network Option
The geometric network in the model is based on close distances. While many people do indeed interact most with those closest to them physically, modern life does include a lot of movement between communities. Starting with the spatially-clustered network in the model, add a new network procedure that randomly rewires a specified number of links, and add a slider to set the % of links to rewire as well as a toggle were the user can select which network to use.


## Step 2: Network Measures
Add at least two reporters showing informative network measures. You are welcome to write them from scratch or use the NetLogo Nw extension primitives. The documentation for the extension can be found here: https://ccl.northwestern.edu/netlogo/docs/nw.htmlLinks to an external site.

To use an extension you must include this at the beginning of your code:  

extensions [ nw ]

; this command will return the mean patch length for the whole model
nw:mean-path-length
For an example of how to use the nw extension to calculate network measures, there is an example Netlogo model located in the Models Library that comes with Netlogo. It is located in Code Examples, Extension Examples, NW, NW General Examples.

How are the networks at various numbers of new links similar and dissimilar? 

## Step 3: Sensitivity Analysis
Perform a sensitivity analysis on your new version. The first task in a sensitivity analysis is determining the appropriate number of runs. The model has been run 10,000 times and the graphs below have been created from your results (These graphs are included to make the assignment simpler, but since they are not from your model the actual number of runs needed could be different). Use these graphs to determine the appropriate number of runs:

sus_cofv.png

resistant_cofv.png

Select appropriate number of runs, parameters (especially your re-wiring parameter) and analysis currencies, paying special attention to the sensitivity of the model to network type and whether or not superspreaders exist.

## Step 1

The modification adds a small feature allowing the user to control additional (approximately) random link-generation with a toggle and sliders. The new controls are located under the grid display. New code is located in a block demarcated with my name. Further information about the modification appears below.

## Step 2

I was unable to get the `nw` extension to work at all with my setup. Instead, I added a very basic display for `links-per-node`. A also borrowed this code:

```netlogo
if not plot? [ stop ]
let max-degree max [count link-neighbors] of turtles
plot-pen-reset  ;; erase what we plotted before
set-plot-x-range 1 (max-degree + 1)  ;; + 1 to make room for the width of the last bar
histogram [count link-neighbors] of turtles
```

...from the library model, Preferential Attachment, to generate a histogram plot of the network Degree Distribution. These plots are particularly sensitive to activity from the customization mentioned in Step 1, since additional links can be added over time.

## Step 3

From the supplied images it appears CV stabilizes at around 100-200 runs for both the susceptable and resistant currencies. I observe that the included "BehaviorSpace" setup has this number at 100, and so we will use that run-number for our experiment (called "Planes" in the .nlogo file.) Weʻll expect to use those same currencies, though we might also peak in on "dead" (survival) as well.

The given variables for BehaviorSpace from the March10 experiment were:

```
["test-frequency" 2 7 14]
["initial-outbreak-size" 10]
["transmitter" true false]
["average-degree" 3 6]
["quarantine-time" 0 7 14]
["morbidity" 10]
["max-recovery-time" 14]
["transmitter-spread" 90]
["transmitter-frequency" 10]
["population" 300]
["virus-spread-chance" 40]
```

It is possible that some of these variables have changed since that version. I will use the following substitutes:

```
["test-frequency" 2 7 14]
["initial-outbreak-size" 10]
["average-degree" 3]
["quarantine-time" 0 7 14]
["morbidity" 10]
["max-recovery-time" 21]
["spreader-reproduction" 90]
["spreader-frequency" 10 25 40]
["non-spreader-reproduction" 10]
["population" 300]
["reproduction" 20]
```

I will add to these variables the following modification-specific variables:

```
["planes-on?" true]
["plane-frequency" 5 15 25]
["plane-radius" 1 2]
```

With the runs complete, all data are located in `M6Model_Dresslar-Planes-table.csv`.

In [16]:
import pandas as pd
import numpy as np

csv_file = "M6Model_Dresslar-Planes-table.csv"

# read datafram in but skip 6 rows of headers
df = pd.read_csv(csv_file, skiprows=6)

# Parameters we want to analyze
params_of_interest = ["plane-frequency", "plane-radius", "spreader-frequency", "quarantine-time", "test-frequency"]

currency_cols = ['susceptable', 'resistant', 'dead', 'quarantine'] # infected is not include since it is automatically 0

# print(df.head())  # note that this is now ALL STEPS. we donʻt want that.

# letʻs create a new df_finals with only the row with the largest value for `step` for each run
# a run is really defined by the combination of all the param_of_interest and the run-number
# just for my benefit weʻll create an "index" column that is a composite of the params_of_interest and the run-number joined by "_"
comp_cols = params_of_interest + ['run-number']
df_finals = df.loc[df.groupby(comp_cols)['step'].idxmax()]
df_finals['index'] = df_finals[comp_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
print(df_finals.shape)
print(df_finals.head())

# now for each param in params_of_interest, we want to show descriptive stats grouped by that param
for param in params_of_interest:
    print(f"\n\n=== Statistics grouped by {param} ===")
    grouped = df_finals.groupby(param)
    for currency in currency_cols:
        print(f"\n-- {currency} --")
        stats = grouped[currency].describe()
        print(stats)






(10800, 22)
      run-number  test-frequency  initial-outbreak-size  average-degree  \
854            1               2                     10               3   
543            2               2                     10               3   
1039           3               2                     10               3   
542            4               2                     10               3   
626            5               2                     10               3   

      quarantine-time  morbidity  max-recovery-time  spreader-reproduction  \
854                 0         10                 21                     90   
543                 0         10                 21                     90   
1039                0         10                 21                     90   
542                 0         10                 21                     90   
626                 0         10                 21                     90   

      spreader-frequency  non-spreader-reproduction  ...  planes-on 

### Description of modification

The new planes modification simulates the expansion of contact networks and the resultant spread of virus through airplane travel. It is a toy modification and its sophistication is utterly minimal.

A second breed of turtles has been added, reclassing the existing turtles as `nodes` and the new breed as `planes`. If the `planes-on?` toggle is set to On, planes will randomly emanate from one of the four "walls" of the arena and head on a random vector to some other wall. Along the way, any nodes within `plane-radius` of the plane are connected to *every other node* the plane has already come into contact with, following the linking rules of the base model (for instance, "dead" nodes are not linked). Higher values of `plane-radius` can thus generate very large numbers of network connections.

The model otherwise functions as normal, and disease can be seen to travel across these new, airplane-generated links, just as would be expected with the real world phenomenon.